In [ ]:
# Code repurposed for testing from 
#https://www.kaggle.com/kazmiekr/predicting-nhl-hall-of-famers/comments

In [2]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import imblearn
from imblearn.ensemble import BalancedRandomForestClassifier


%matplotlib inline
import tensorflow as tf

ImportError: cannot import name 'abs'

In [4]:
# master_df = pd.read_csv('Jeff/Output/test_train_file.csv')
#1967 onwards with skaters with >100 GP
master_df = pd.read_csv('Jeff/Output/tt_filt67_100GP_file.csv')
master_df.head()

,Player,GP,G,A,TP,PPG,PIM,Years,POS,firstYear,...,Art Ross Awards,Conn Smythe Awards,Hart Awards,Calder Awards,James Norris Awards,Ted Lindsay Awards,Maurice Richard Awards,Cups,All-Star Games,HoF
0,Wayne Gretzky,1487,894,1963,2857,1.921,577,20,1,1979,...,10,2,9,0,0,5,0,4,18,1
1,Mark Messier,1756,694,1193,1887,1.075,1910,25,1,1979,...,0,1,2,0,0,2,0,0,15,1
2,Gordie Howe,1767,801,1049,1850,1.047,1685,26,1,1946,...,6,0,6,0,0,0,0,4,23,1
3,Ron Francis,1731,549,1249,1798,1.039,979,23,1,1981,...,0,0,0,0,0,0,0,2,4,1
4,Marcel Dionne,1348,731,1040,1771,1.314,600,18,1,1971,...,1,0,0,0,0,2,0,0,8,1


In [5]:
master_df.columns

Index(['Player', 'GP', 'G', 'A', 'TP', 'PPG', 'PIM', 'Years', 'POS',
       'firstYear', 'lastYear', 'Art Ross Awards', 'Conn Smythe Awards',
       'Hart Awards', 'Calder Awards', 'James Norris Awards',
       'Ted Lindsay Awards', 'Maurice Richard Awards', 'Cups',
       'All-Star Games', 'HoF'],
      dtype='object')

In [6]:
new_hof = ['Jarome Iginla', 'Marian Hossa','Martin St. Louis','Teemu Selanne','Nicklas Lidstrom','Chris Pronger','Mark Recchi','Mike Modano','Peter Forsberg']
new_hof

['Jarome Iginla',
 'Marian Hossa',
 'Martin St. Louis',
 'Teemu Selanne',
 'Nicklas Lidstrom',
 'Chris Pronger',
 'Mark Recchi',
 'Mike Modano',
 'Peter Forsberg']

In [7]:
#drop hof additions since 2011 for testing purposes
inverse_boolean_series = ~master_df.Player.isin(new_hof)
retired_players = master_df[inverse_boolean_series]

retired_players = retired_players[retired_players['GP']>=100]

#create df with only new hof additions for testing
boolean_series = master_df.Player.isin(new_hof)
new_hof_df = master_df[boolean_series]
# print(new_hof_df)


initial_eligible_players = pd.read_csv('Jeff/Output/predict_file.csv')

#isolate any 'active' hof in the eligible players dataset
active_hof = initial_eligible_players[initial_eligible_players['HoF'] == 1]
# print(active_hof)
#append any 'active' hof players to the test set
new_hof_df = new_hof_df.append(active_hof)

#update the eligible players df
eligible_players = initial_eligible_players[initial_eligible_players['HoF'] == 0]
# print(new_hof_df)

In [8]:
cols_to_drop = ['Player','firstYear', 'lastYear','POS', 'HoF']

In [9]:
# print(new_hof_df)

In [10]:
#Training Set
train=retired_players.sample(frac=0.70, random_state=16)
# print(train.shape)

#Test Set
test=retired_players.drop(train.index)
# print(test.shape)

X_train = train.drop(cols_to_drop, axis=1)
Y_train = train['HoF']
X_test = test.drop(cols_to_drop, axis=1).copy()
Y_test = test['HoF']

# Dataset with unknown HoF predictions
X_player_test = eligible_players.drop(cols_to_drop, axis=1).copy()

# Create a test set of players that have made the HF since the last year of available data in this data set
# We'll just fill the Y with 1's
X_new_hof_test = new_hof_df.drop(cols_to_drop, axis=1).copy()
Y_new_hof_test = np.ones(X_new_hof_test.shape[0], dtype=np.int)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape


((1750, 16), (1750,), (750, 16), (750,))

In [11]:
classifiers = {}
# Run a classifier instance against the training set
def run_classifier(classifier):
    # Get the name of the classifier instance
    class_name = classifier.__class__.__name__
    print("Testing {0}".format(class_name))
    print("----------------------------------------\n")
    
    # Fit the classifier to the training set
    classifier.fit(X_train, Y_train)
    # Find out the training accuracy
    training_accuracy = round(classifier.score(X_train, Y_train) * 100, 2)
    print("Training Score: {0}".format(training_accuracy))
    
    # Use the trained data to predict the next HoF's
    Y_pred = classifier.predict(X_test)
    
    # How accurate was it against the known set of recent HoF's
    hof_accuracy = test_current_hf(classifier)
    
    # Cross validation scores
    f1_score = f1(Y_test, Y_pred)
    
    
    print("\n----- Predictions -----\n")
    # For all the classifiers except LinearSVC, we'll determine the probability in addition
    # to the prediction
    if class_name != 'LinearSVC':
        (positive_predictions, _) = show_prediction_with_probabilities(classifier, X_player_test)
    else:
        (positive_predictions, _) = show_prediction(classifier, X_player_test)
        
    # Store our results per classifier for later analysis    
    classifiers[class_name] = {
        'Training Accuracy': training_accuracy,
        'F1 Score': f1_score,
        'Current HoF Accuracy': hof_accuracy
    }
    return positive_predictions

# Given our set of known recent HoF additions, does the classifier think they should have made it?
def test_current_hf(classifier):
    accuracy = round(classifier.score(X_new_hof_test, Y_new_hof_test) * 100, 2)
    print("Accuracy with Current HF: {0}".format(accuracy))
    return accuracy

# Display from cross validation scores, F1, precision, recall
def f1(Y_test, Y_pred):
    macro = f1_score(Y_test.values, Y_pred, average='macro')
    print("F1 Score: {0}".format(formatPercent(macro)))
    precision = precision_score(Y_test.values, Y_pred, average='macro')
    print("Precision Score: {0}".format(formatPercent(precision)))
    recall = recall_score(Y_test.values, Y_pred, average='macro')
    print("Recall Score: {0}".format(formatPercent(recall)))
    return macro

def formatPercent(num):
    return round(num, 2)
    
def show_prediction_with_probabilities(classifier, X_player_test, display=True):
    prediction = classifier.predict(X_player_test)
    prob = classifier.predict_proba(X_player_test)
    pred_df = pd.DataFrame({
#         "playerID": eligible_players["playerID"],
        "Player": eligible_players["Player"],
        "isInHfPrediction": prediction,
        #"prob1": prob[:,0],
        "probability": prob[:,1]
    })
    positive_predictions = pred_df[pred_df['isInHfPrediction'] == 1]
    if display == True:
        print(positive_predictions.sort_values(by='probability', ascending=False))    
    return positive_predictions, pred_df

def show_prediction(classifier, X_player_test):
    prediction = classifier.predict(X_player_test)
    pred_df = pd.DataFrame({
#         "playerID": eligible_players["playerID"],
        "Player": eligible_players["Player"],
        "isInHfPrediction": prediction
    })
    positive_predictions = pred_df[pred_df['isInHfPrediction'] == 1]
    print(positive_predictions)
    return positive_predictions, pred_df

In [12]:
nn = MLPClassifier(alpha=1)
nn_pred = run_classifier(nn)

Testing MLPClassifier
----------------------------------------

Training Score: 96.57
Accuracy with Current HF: 100.0
F1 Score: 0.73
Precision Score: 0.69
Recall Score: 0.77

----- Predictions -----

                 Player  isInHfPrediction  probability
0          Jaromir Jagr                 1     1.000000
1          Joe Thornton                 1     0.999992
3         Sidney Crosby                 1     0.999988
8          Patrick Kane                 1     0.999964
2    Alexander Ovechkin                 1     0.999425
54        Erik Karlsson                 1     0.998635
5        Yevgeni Malkin                 1     0.998013
15    Nicklas Backstrom                 1     0.998011
20        Claude Giroux                 1     0.997300
11         Ryan Getzlaf                 1     0.996396
101      Connor McDavid                 1     0.994416
6          Henrik Sedin                 1     0.988023
13         Anze Kopitar                 1     0.980241
19       Steven Stamkos       

In [13]:
logreg = LogisticRegression()
logreg_pred = run_classifier(logreg)

Testing LogisticRegression
----------------------------------------

Training Score: 97.94
Accuracy with Current HF: 77.78
F1 Score: 0.81
Precision Score: 0.83
Recall Score: 0.78

----- Predictions -----

                 Player  isInHfPrediction  probability
0          Jaromir Jagr                 1     0.999961
8          Patrick Kane                 1     0.999731
3         Sidney Crosby                 1     0.985074
2    Alexander Ovechkin                 1     0.971285
5        Yevgeni Malkin                 1     0.954847
1          Joe Thornton                 1     0.944028
13         Anze Kopitar                 1     0.902093
52         Duncan Keith                 1     0.898334
88         Drew Doughty                 1     0.839884
54        Erik Karlsson                 1     0.836059
9            Eric Staal                 1     0.822302
22       Jonathan Toews                 1     0.810604
47         Tyler Seguin                 1     0.798471
76          Kris Letang  

C:\Users\Y243863\Anaconda3\envs\bcs\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [14]:
svc = SVC(probability=True)
svc_pred = run_classifier(svc)

Testing SVC
----------------------------------------

Training Score: 97.77
Accuracy with Current HF: 66.67
F1 Score: 0.76
Precision Score: 0.86
Recall Score: 0.71

----- Predictions -----

               Player  isInHfPrediction  probability
1        Joe Thornton                 1     0.999984
0        Jaromir Jagr                 1     0.995746
3       Sidney Crosby                 1     0.986874
2  Alexander Ovechkin                 1     0.964263
6        Henrik Sedin                 1     0.843032
4     Patrick Marleau                 1     0.754232
5      Yevgeni Malkin                 1     0.708477


In [15]:
coeff_df = pd.DataFrame(retired_players.columns.drop(['Player','firstYear', 'lastYear','POS', 'HoF']))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
15,All-Star Games,0.965478
14,Cups,0.932154
11,James Norris Awards,0.457996
8,Conn Smythe Awards,0.299935
7,Art Ross Awards,0.222544
12,Ted Lindsay Awards,0.107151
3,TP,0.102167
9,Hart Awards,0.084716
10,Calder Awards,0.074345
0,GP,0.003690


In [16]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn_pred = run_classifier(knn)

Testing KNeighborsClassifier
----------------------------------------

Training Score: 98.23
Accuracy with Current HF: 66.67
F1 Score: 0.77
Precision Score: 0.82
Recall Score: 0.73

----- Predictions -----

                Player  isInHfPrediction  probability
0         Jaromir Jagr                 1     1.000000
1         Joe Thornton                 1     1.000000
2   Alexander Ovechkin                 1     1.000000
3        Sidney Crosby                 1     1.000000
6         Henrik Sedin                 1     1.000000
4      Patrick Marleau                 1     0.666667
5       Yevgeni Malkin                 1     0.666667
8         Patrick Kane                 1     0.666667
12   Henrik Zetterberg                 1     0.666667
13        Anze Kopitar                 1     0.666667
14        Jason Spezza                 1     0.666667
43         Zdeno Chara                 1     0.666667


In [17]:
gaussian = GaussianNB()
gaussian_pred = run_classifier(gaussian)

Testing GaussianNB
----------------------------------------

Training Score: 97.43
Accuracy with Current HF: 77.78
F1 Score: 0.79
Precision Score: 0.75
Recall Score: 0.87

----- Predictions -----

                 Player  isInHfPrediction  probability
0          Jaromir Jagr                 1     1.000000
40          Brent Burns                 1     1.000000
134          Roman Josi                 1     1.000000
126         P.K. Subban                 1     1.000000
125      Leon Draisaitl                 1     1.000000
101      Connor McDavid                 1     1.000000
100       Victor Hedman                 1     1.000000
94        Mark Giordano                 1     1.000000
88         Drew Doughty                 1     1.000000
74      Nikita Kucherov                 1     1.000000
68        Ryan O'Reilly                 1     1.000000
1          Joe Thornton                 1     1.000000
54        Erik Karlsson                 1     1.000000
52         Duncan Keith          

In [18]:
linear_svc = LinearSVC()
linear_svc_pred = run_classifier(linear_svc)

Testing LinearSVC
----------------------------------------

Training Score: 97.43
Accuracy with Current HF: 66.67
F1 Score: 0.77
Precision Score: 0.84
Recall Score: 0.73

----- Predictions -----

                 Player  isInHfPrediction
0          Jaromir Jagr                 1
1          Joe Thornton                 1
2    Alexander Ovechkin                 1
3         Sidney Crosby                 1
5        Yevgeni Malkin                 1
6          Henrik Sedin                 1
8          Patrick Kane                 1
11         Ryan Getzlaf                 1
13         Anze Kopitar                 1
15    Nicklas Backstrom                 1
19       Steven Stamkos                 1
20        Claude Giroux                 1
30         John Tavares                 1
54        Erik Karlsson                 1
74      Nikita Kucherov                 1
90     Nathan MacKinnon                 1
101      Connor McDavid                 1
111     Johnny Gaudreau                 1


C:\Users\Y243863\Anaconda3\envs\bcs\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [19]:
decision_tree = DecisionTreeClassifier()
decision_tree_pred = run_classifier(decision_tree)

Testing DecisionTreeClassifier
----------------------------------------

Training Score: 100.0
Accuracy with Current HF: 88.89
F1 Score: 0.78
Precision Score: 0.81
Recall Score: 0.76

----- Predictions -----

                 Player  isInHfPrediction  probability
0          Jaromir Jagr                 1          1.0
2    Alexander Ovechkin                 1          1.0
134          Roman Josi                 1          1.0
126         P.K. Subban                 1          1.0
100       Victor Hedman                 1          1.0
94        Mark Giordano                 1          1.0
90     Nathan MacKinnon                 1          1.0
88         Drew Doughty                 1          1.0
76          Kris Letang                 1          1.0
54        Erik Karlsson                 1          1.0
52         Duncan Keith                 1          1.0
50        Patrick Sharp                 1          1.0
40          Brent Burns                 1          1.0
30         John Tavar

In [20]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest_pred = run_classifier(random_forest)

Testing RandomForestClassifier
----------------------------------------

Training Score: 100.0
Accuracy with Current HF: 100.0
F1 Score: 0.83
Precision Score: 0.93
Recall Score: 0.77

----- Predictions -----

                Player  isInHfPrediction  probability
0         Jaromir Jagr                 1         1.00
2   Alexander Ovechkin                 1         0.93
1         Joe Thornton                 1         0.87
8         Patrick Kane                 1         0.85
3        Sidney Crosby                 1         0.81
43         Zdeno Chara                 1         0.76
5       Yevgeni Malkin                 1         0.68
54       Erik Karlsson                 1         0.66
4      Patrick Marleau                 1         0.62
40         Brent Burns                 1         0.60
19      Steven Stamkos                 1         0.55
6         Henrik Sedin                 1         0.54
9           Eric Staal                 1         0.51


In [21]:
model_performance = pd.DataFrame.from_dict(classifiers, orient='index')
model_performance.sort_values(by='F1 Score', ascending=False)

,Training Accuracy,F1 Score,Current HoF Accuracy
RandomForestClassifier,100.00,0.828532,100.00
LogisticRegression,97.94,0.806302,77.78
GaussianNB,97.43,0.793965,77.78
DecisionTreeClassifier,100.00,0.784780,88.89
LinearSVC,97.43,0.772550,66.67
KNeighborsClassifier,98.23,0.765859,66.67
SVC,97.77,0.761781,66.67
MLPClassifier,96.57,0.726613,100.00


In [22]:
gaussian_pred.sort_values(by='probability', ascending=False).head(20)

,Player,isInHfPrediction,probability
0,Jaromir Jagr,1,1.0
40,Brent Burns,1,1.0
134,Roman Josi,1,1.0
126,P.K. Subban,1,1.0
125,Leon Draisaitl,1,1.0
101,Connor McDavid,1,1.0
100,Victor Hedman,1,1.0
94,Mark Giordano,1,1.0
88,Drew Doughty,1,1.0
74,Nikita Kucherov,1,1.0
